# Mini Project BGP 3
# Assignment 2 Regression

William Arthurius & Annisa Rizqilana
October 2024

# Intializing

In [1]:
import ee

# Trigger the authentication flow.
#ee.Authenticate()

In [2]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


# Getting Data

In [3]:
import geemap
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [5]:
#fc = ee.FeatureCollection(Map.draw_last_feature)
#region = fc.geometry([[[36.644321, -1.490252],
#  [36.644321, -1.153886],
#  [37.109971, -1.153886],
#  [37.109971, -1.490252],
#  [36.644321, -1.490252]]])
#cords = region.getInfo()['coordinates']
#cords

In [4]:
# # or alternatively use the following predefined rectangle
region = ee.Geometry.Polygon(
[[[36.644321, -1.490252],
  [36.644321, -1.153886],
  [37.109971, -1.153886],
  [37.109971, -1.490252],
  [36.644321, -1.490252]]]
 )

In [5]:
myCollection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(region) \
    .filterDate('2020-01-01', '2020-12-31') \
    .filterMetadata('CLOUD_COVER', 'less_than', 10)

In [6]:
# List the cloud cover percentage for each image in the collection
cloud_info = myCollection.aggregate_array('CLOUD_COVER').getInfo()
image_ids = myCollection.aggregate_array('system:index').getInfo()

# Print the cloud cover for each image
for img_id, cloud_cover in zip(image_ids, cloud_info):
    print(f"Image ID: {img_id}, Cloud Cover: {cloud_cover}%")

Image ID: LC08_168061_20200220, Cloud Cover: 1.32%
Image ID: LC08_168061_20200830, Cloud Cover: 3.27%


In [7]:
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))
listOfImages

Number of images in the collection:  2


['LC08_168061_20200220', 'LC08_168061_20200830']

In [8]:
# Visualize the first Landsat Map

img1 = myCollection.first().clip(region)
vis_params = {"min": 0, 
              "max": 15000, 
              "bands": ["SR_B4", "SR_B3", "SR_B2"]}  
Map2 = geemap.Map()
Map2.addLayer(img1, vis_params, "FirstImage", True) 
Map2

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# NDVI

In [9]:
# Function to calculate NDVI on a monthly composite
def calculate_ndvi_for_month(year, month):
    # Define start and end dates for the month
    start_date = ee.Date(f"{year}-{month:02d}-01")
    end_date = start_date.advance(1, 'month')
    
    # Filter images within the date range and calculate the median composite
    monthly_composite = myCollection.filterDate(start_date, end_date).median()
    
    # Calculate NDVI from the median composite
    ndvi = monthly_composite.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return ndvi.set('month', f"{year}-{month:02d}")

# Calculate NDVI for February and August 2020
ndvi_february = calculate_ndvi_for_month(2020, 2)
ndvi_august = calculate_ndvi_for_month(2020, 8)

In [10]:
# Visualization parameters for NDVI
vis_params_ndvi = {
    'min': -0.1,
    'max': 0.6,
    'palette': ['blue', 'white', 'green']
}

# Add February and August NDVI layers to the map
Map.addLayer(ndvi_february.clip(region), vis_params_ndvi, "NDVI February 2020")
Map.addLayer(ndvi_august.clip(region), vis_params_ndvi, "NDVI August 2020")

# Display the map
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

# Elevation

In [15]:
# Step 1: Load the SRTM elevation data
srtm_elevation = ee.Image("USGS/SRTMGL1_003").select("elevation").clip(region)

# Display elevation on the map (optional)
elevation_vis_params = {
    "min": 0,
    "max": 3000,
    "palette": ["blue", "green", "yellow", "brown", "white"]
}
Map.addLayer(srtm_elevation, elevation_vis_params, "Elevation (SRTM)", True)
Map

Map(bottom=132335.0, center=[-1.3220761135867312, 36.877145999999826], controls=(WidgetControl(options=['posit…

# Precipitation

In [16]:
# Function to calculate monthly precipitation by summing daily values
def calculate_monthly_precipitation(year, month):
    start_date = ee.Date(f"{year}-{month:02d}-01")
    end_date = start_date.advance(1, 'month')
    
    # Sum daily precipitation within the month
    precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
        .filterDate(start_date, end_date) \
        .sum() \
        .rename("precipitation")
    
    return precipitation.set('month', f"{year}-{month:02d}")

# Calculate precipitation for February and August 2020
precipitation_february = calculate_monthly_precipitation(2020, 2)
precipitation_august = calculate_monthly_precipitation(2020, 8)

In [17]:
# Visualization parameters for precipitation
vis_params_precip = {
    'min': 0,
    'max': 300,
    'palette': ['lightblue', 'blue', 'darkblue']
}

# Visualize February and August precipitation on the map
Map.addLayer(precipitation_february.clip(region), vis_params_precip, "Precipitation February 2020")
Map.addLayer(precipitation_august.clip(region), vis_params_precip, "Precipitation August 2020")

# Display the map with precipitation layers
Map

Map(bottom=132335.0, center=[-1.3220761135867312, 36.877145999999826], controls=(WidgetControl(options=['posit…

# Data Sampling 

In [18]:
# Generate 1000 random points within the region
random_points = ee.FeatureCollection.randomPoints(region, 1000, seed=42)

# Display the random points on the map
Map.addLayer(random_points, {}, "Random Points")
Map.centerObject(region)

Map

Map(bottom=132335.0, center=[-1.3220761135867312, 36.877145999999826], controls=(WidgetControl(options=['posit…

In [21]:
# Create combined image with all predictor and target bands
combined_image = (
    ndvi_february.rename("NDVI_February")
    .addBands(ndvi_august.rename("NDVI_August"))
    .addBands(srtm_elevation.rename("Elevation"))
    .addBands(precipitation_february.rename("Precipitation_February"))
    .addBands(precipitation_august.rename("Precipitation_August"))
)

In [23]:
# Sample the combined data for all bands
sampled_data = combined_image.sampleRegions(
    collection=random_points,
    scale=30,
    geometries=True
)

In [24]:
# Export the sampled data to CSV using geemap
output_csv = './sampled_data.csv'
geemap.ee_export_vector(sampled_data, filename=output_csv)

print(f"Sampled data exported to {output_csv}")

Generating URL ...
Please wait ...
Data downloaded to /data/private/BGP/sampled_data.csv
Sampled data exported to ./sampled_data.csv


# Data Cleaning

In [26]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('sampled_data.csv')

In [27]:
data.head()

,Precipitation_February,Elevation,NDVI_August,Precipitation_August,NDVI_February,system:index
0,72.970635,1888,0.226761,16.290746,0.359078,0_0
1,51.921231,1614,0.151698,6.716886,0.195440,1_0
2,62.431686,1741,0.187869,16.729264,0.266376,2_0
3,70.412409,1677,0.237223,12.633935,0.331406,3_0
4,80.160094,1594,0.082195,10.396948,0.125478,4_0


In [31]:
# Rename the column for clarity (optional)
data = data.rename(columns={
    "system:index": "ID",
    "NDVI_February": "NDVI_February",
    "Precipitation_February": "Precipitation_February",
    "Elevation": "Elevation",
    "NDVI_August": "NDVI_August",
    "Precipitation_August": "Precipitation_August"
})

# Ensure the data is in wide format, with each ID having columns for February and August values side-by-side
data_wide = data[[
    "ID", 
    "Elevation", 
    "NDVI_February", "Precipitation_February", 
    "NDVI_August", "Precipitation_August"
]]

# Display the reshaped data
print(data_wide.head())

    ID  Elevation  NDVI_February  Precipitation_February  NDVI_August  \
0  0_0       1888       0.359078               72.970635     0.226761   
1  1_0       1614       0.195440               51.921231     0.151698   
2  2_0       1741       0.266376               62.431686     0.187869   
3  3_0       1677       0.331406               70.412409     0.237223   
4  4_0       1594       0.125478               80.160094     0.082195   

   Precipitation_August  
0             16.290746  
1              6.716886  
2             16.729264  
3             12.633935  
4             10.396948  


In [33]:
# Reshape the DataFrame to long format
data_long = pd.DataFrame({
    "ID": data["ID"].tolist() * 2,  # Repeat each ID twice (once for each month)
    "Month": ["February"] * len(data) + ["August"] * len(data),  # Month column for February and August
    "Elevation": data["Elevation"].tolist() * 2,  # Repeat elevation values for both months
    "NDVI": data["NDVI_February"].tolist() + data["NDVI_August"].tolist(),  # NDVI values for both months
    "Precipitation": data["Precipitation_February"].tolist() + data["Precipitation_August"].tolist()  # Precipitation values for both months
})

# Sort by ID and Month, ensuring February comes before August for each ID
data_long["Month"] = pd.Categorical(data_long["Month"], categories=["February", "August"], ordered=True)
data_long = data_long.sort_values(by=["ID", "Month"]).reset_index(drop=True)

# Display the reordered data
print(data_long.head(20))

       ID     Month  Elevation      NDVI  Precipitation
0     0_0  February       1888  0.359078      72.970635
1     0_0    August       1888  0.226761      16.290746
2   100_0  February       1555  0.250845      50.103367
3   100_0    August       1555  0.130315       5.156040
4   101_0  February       1608  0.255370      50.392181
5   101_0    August       1608  0.160356       9.876567
6   102_0  February       1498  0.332940      63.053939
7   102_0    August       1498  0.172909       0.000000
8   103_0  February       1688  0.367190      43.796728
9   103_0    August       1688  0.122736       6.106490
10  104_0  February       1582  0.240739      80.160094
11  104_0    August       1582  0.136137      10.396948
12  105_0  February       1519  0.221231      62.510326
13  105_0    August       1519  0.124342       5.146584
14  106_0  February       1857  0.341764      71.403581
15  106_0    August       1857  0.206505      22.460916
16  107_0  February       1491  0.274221      58

# Regression 

In [44]:
# Select features (predictors) and target
X = data_long[['Elevation', 'Precipitation']]
y = data_long['NDVI']

In [45]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [46]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on the training data, then transform both training and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [90]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Initialize the XGBoost Regressor
xgbr = XGBRegressor(n_estimators=50, subsample=1.0, learning_rate=0.3, max_depth=7, colsample_bytree=0.8, random_state=42)

# Train the model
xgbr.fit(X_train, y_train)

# Predict on the test set
y_pred_xgb = xgbr.predict(X_test)

# Calculate evaluation metrics
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

# Display the results
print("XGBoost Regressor - Mean Absolute Error (MAE):", mae_xgb)
print("XGBoost Regressor - R-squared (R²):", r2_xgb)

XGBoost Regressor - Mean Absolute Error (MAE): 0.04448416179039577
XGBoost Regressor - R-squared (R²): 0.5412103856930204


In [87]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# Expanded parameter grid for XGBoost
param_grid_xgb = {
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.2, 0.3],
    'max_depth': [3, 5, 7, 10, 15],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'reg_alpha': [0, 0.1, 0.5],  # L1 regularization
    'reg_lambda': [1, 1.5, 2]     # L2 regularization
}

# Initialize XGBoost
xgb = XGBRegressor(random_state=42)

# Randomized search with cross-validation
random_search_xgb = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid_xgb,
    n_iter=1000,  # Number of random combinations to try
    scoring='r2',
    cv=5,
    random_state=42,
    n_jobs=-1,
    verbose=2
)

# Fit the model
random_search_xgb.fit(X_train, y_train)

# Display best parameters and R-squared score
print("Best parameters for XGBoost:", random_search_xgb.best_params_)
print("Best R-squared score from RandomizedSearchCV:", random_search_xgb.best_score_)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
[CV] END colsample_bytree=0.8, learning_rate=0.001, max_depth=15, n_estimators=200, subsample=1.0; total time=   0.2s
[CV] END colsample_bytree=0.6, learning_rate=0.2, max_depth=5, n_estimators=100, subsample=0.6; total time=   0.1s
[CV] END colsample_bytree=0.6, learning_rate=0.001, max_depth=10, n_estimators=500, subsample=0.8; total time=   0.4s
[CV] END colsample_bytree=0.6, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.3, max_depth=7, n_estimators=200, subsample=0.6; total time=   0.3s
[CV] END colsample_bytree=0.6, learning_rate=0.05, max_depth=10, n_estimators=300, subsample=0.8; total time=   0.5s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=15, n_estimators=200, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=0.6, learning_rate=0.3, max_depth=7, n_estimators=300, subsample=1.0; total time=   0.3s
[CV] 